In [ ]:
import gradio as gr
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity

#fce pro získání CLIP embeddingu
def get_clip_embeddings(input_data, model, processor, input_type='text'):
    if input_type == 'text':
        inputs = processor(text=input_data, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            embeddings = model.get_text_features(**inputs)
    elif input_type == 'image':
        if isinstance(input_data, str):
            image = Image.open(input_data)
        inputs = processor(images=image, return_tensors="pt")
        with torch.no_grad():
            embeddings = model.get_image_features(**inputs)
    return embeddings.numpy()

# načtení modelu a processoru
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# načtení embeddingů obrázků (pro ukázku zmenšená sada 200 obrázků)
image_dir = "cesta k sample image"
image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# počítání embeddingů pro všechny obrázky
image_embeddings = []
for image_path in image_paths:
    embedding = get_clip_embeddings(image_path, model, processor, input_type='image')
    image_embeddings.append(embedding)

image_embeddings = np.vstack(image_embeddings)

# fce pro Gradio aplikaci
def find_similar_images(text_input):
    # Získání embeddingu pro text
    text_embedding = get_clip_embeddings(text_input, model, processor, input_type='text')

    # Výpočet kosinové podobnosti mezi textem a obrázky
    similarities = cosine_similarity(text_embedding, image_embeddings)

    # Seřazení podle podobnosti
    best_indices = np.argsort(similarities[0])[::-1][:4]

    # Výběr nejlepších 4 obrázků
    best_images = [image_paths[i] for i in best_indices]
    return [Image.open(img) for img in best_images]

# vytvoření Gradio rozhraní
interface = gr.Interface(
    fn=find_similar_images,
    inputs="text",
    outputs=[gr.Image(type="pil")] * 4,
    title="Find Similar Images with CLIP",
    description="Enter a text prompt to find the most similar images."
)

# spuštění aplikace
interface.launch() 